In [1]:
#Rhea's version
%cd /content
%rm -rf NASLib
!git clone -b Develop_copy https://github.com/automl/NASLib/
%cd /content/NASLib
!pip install -e . --quiet
!pip uninstall --yes matplotlib
!pip install matplotlib==2.1.1
exit()
# The runtime would be automatically restarted now. Just proceed with running the next cell now

/content
Cloning into 'NASLib'...
remote: Enumerating objects: 31761, done.
remote: Counting objects: 100% (1717/1717), done.
remote: Compressing objects: 100% (781/781), done.
remote: Total 31761 (delta 1034), reused 1575 (delta 911), pack-reused 30044
Receiving objects: 100% (31761/31761), 549.06 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (22437/22437), done.
/content/NASLib
     |████████████████████████████████| 636 kB 7.6 MB/s 
     |████████████████████████████████| 831.4 MB 2.1 kB/s 
     |████████████████████████████████| 22.1 MB 54.9 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 11.2 MB 34.8 MB/s 
     |████████████████████████████████| 280 kB 57.6 MB/s 
     |████████████████████████████████| 2.0 MB 65.8 MB/s 
     |████████████████████████████████| 166.7 MB 16 kB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 1.6 MB 35.2 MB/s 
     |███████████████████████████████

In [1]:
# Import the search space
from naslib.search_spaces import NasBench201SearchSpace
# Sample a random architecture
# You can call this method only once.
graph = NasBench201SearchSpace(n_classes=10)
graph.sample_random_architecture()

# Get the NASLib representation of this architecture
graph.get_hash()

/usr/local/lib/python3.7/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


(0, 4, 1, 0, 4, 4)

In [3]:
## This graph is now a NAS-Bench-201 model, which can be used for training
# Forward pass some dummy data through it to see it in action

import torch

x = torch.randn(5, 3, 32, 32) # (Batch_size, Num_channels, Height, Width)
logits = graph(x)

print(logits.shape)

torch.Size([5, 10])


In [4]:
# Import code to convert NASLib graph to the original NAS-Bench-201 representation
from naslib.search_spaces.nasbench201.conversions import convert_naslib_to_str as convert_naslib_nb201_to_str

# Get the string representation of this model
convert_naslib_nb201_to_str(graph)

'|skip_connect~0|+|avg_pool_3x3~0|skip_connect~1|+|none~0|avg_pool_3x3~1|avg_pool_3x3~2|'

In [5]:
# Mutating an architecture
# First, create a new child_graph
child_graph = NasBench201SearchSpace(n_classes=10)

# Call mutate on the child graph by passing the parent graph to it
child_graph.mutate(parent=graph)

# See the parent and child graph representations
print(f'Parent graph: {graph.get_hash()}')
print(f'Child graph : {child_graph.get_hash()}')

Parent graph: (0, 4, 1, 0, 4, 4)
Child graph : (0, 4, 1, 0, 3, 4)


In [14]:
%cd NASLib
!source /content/NASLib/scripts/bash_scripts/download_benchmarks.sh nb201 cifar10

/content/NASLib
dataset = cifar10
search_space = nb201
Downloading...
From: https://drive.google.com/uc?id=1sh8pEhdrgZ97-VFBVL94rI36gedExVgJ
To: /content/NASLib/naslib/data/nb201_cifar10_full_training.pickle
100% 117M/117M [00:00<00:00, 178MB/s]


In [15]:
# Now, let's load the queryable tabular NAS-Bench-201 API
# This API has the training metrics of all the 15625 models in the search space
# such as train and validation accuracies/losses at every epoch

from naslib.utils import get_dataset_api
benchmark_api = get_dataset_api(search_space='nasbench201', dataset='cifar10')

In [16]:
# Now, let's load the queryable tabular NAS-Bench-201 API
# This API has the training metrics of all the 15625 models in the search space
# such as train and validation accuracies/losses at every epoch

from naslib.utils import get_dataset_api
benchmark_api = get_dataset_api(search_space='nasbench201', dataset='cifar10')
# With the NAS-Bench-201 API, we can now query, say, the validation performance of any NB201 model
# Without it, we would have to train the model from scratch to get this information

# First, import the Metric enum
from naslib.search_spaces.core import Metric

# Metric has, among others, these values:
# Metric.TRAIN_ACCURACY
# Metric.VAL_ACCURACY
# Metric.TRAIN_LOSS
# Metric.TEST_LOSS
# Metric.TRAIN_TIME

train_acc_parent = graph.query(metric=Metric.TRAIN_ACCURACY, dataset='cifar10', dataset_api=benchmark_api)
val_acc_parent = graph.query(metric=Metric.VAL_ACCURACY, dataset='cifar10', dataset_api=benchmark_api)

print('Performance of parent model')
print(f'Train accuracy: {train_acc_parent}')
print(f'Validation accuracy: {val_acc_parent}')

# TODO: Query the train and validation performance of the child model
# train_acc_parent = ...
# val_acc_parent = ...

# print('Performance of child model')
# print(f'Train accuracy: {train_acc_child}')
# print(f'Validation accuracy: {val_acc_child}')

Performance of parent model
Train accuracy: 55.53600000366211
Validation accuracy: 53.99


In [17]:
from naslib.search_spaces import NasBench301SearchSpace
from naslib.search_spaces.nasbench301.conversions import convert_naslib_to_genotype as convert_naslib_

In [18]:
import json
import logging
import os

# import the Trainer used to run the optimizer on a given search space
from naslib.defaults.trainer import Trainer
# import the optimizers
from naslib.optimizers import (
    RandomSearch,
    RegularizedEvolution
)
# import the search spaces
from naslib.search_spaces import (
    NasBench101SearchSpace,
    NasBench201SearchSpace,
    NasBench301SearchSpace,
)

from naslib.search_spaces.core.query_metrics import Metric
from naslib import utils
from naslib.utils import get_dataset_api
from naslib.utils.log import setup_logger

from fvcore.common.config import CfgNode # Required to read the config
###### End of imports ######


In [19]:
# The configuration used by the Trainer and Optimizer
config = {
    'dataset': 'cifar10',
    'search': {
        'seed': 0, # 
        'epochs': 5, # Number of epochs (steps) of the optimizer to run
        'fidelity': -1, # 
        'checkpoint_freq': 100,
    },
    'save': 'runs' # folder to save the results to 
}

config = CfgNode.load_cfg(json.dumps(config))

# Make the directories required for search and evaluation
os.makedirs(config['save'] + '/search', exist_ok=True)
os.makedirs(config['save'] + '/eval', exist_ok=True)

# Set up the loggers
logger = setup_logger()
logger.setLevel(logging.INFO)

# See the config
logger.info(f'Configuration is \n{config}')
# logger.info(config)

# Set up the seeds
utils.set_seed(9002)

# Instantiate the search space and get its bench

[10/06 13:41:31 naslib]: Configuration is 
dataset: cifar10
save: runs
search:
  checkpoint_freq: 100
  epochs: 5
  fidelity: -1
  seed: 0


In [20]:
search_space = NasBench201SearchSpace()
dataset_api = get_dataset_api('nasbench201', 'cifar10')

# Instantitate the optimizer and adapt the search space to it
optimizer = RandomSearch(config)
optimizer.adapt_search_space(search_space, dataset_api=dataset_api)

# Create a Trainer
trainer = Trainer(optimizer, config)

# Perform the search
trainer.search(resume_from="", report_incumbent=False)

# Get the results of the search
search_trajectory = trainer.search_trajectory
print('Train accuracies:', search_trajectory.train_acc)
print('Validation accuracies:', search_trajectory.valid_acc)

# Get the validation performance of the best model found in the search phase
best_model_val_acc = trainer.evaluate(dataset_api=dataset_api, metric=Metric.VAL_ACCURACY)
best_model_val_acc

[10/06 13:41:47 nl.defaults.trainer]: param size = 0.000000MB
[10/06 13:41:47 nl.defaults.trainer]: Start training
[10/06 13:41:47 nl.defaults.trainer]: Epoch 0 done. Train accuracy: 99.11600, Validation accuracy: 86.81000
[10/06 13:41:47 nl.defaults.trainer]: Epoch 1 done. Train accuracy: 99.55200, Validation accuracy: 84.74000
[10/06 13:41:47 nl.defaults.trainer]: Epoch 2 done. Train accuracy: 90.96800, Validation accuracy: 84.01000
[10/06 13:41:49 nl.defaults.trainer]: Epoch 3 done. Train accuracy: 99.16800, Validation accuracy: 84.36000
[10/06 13:41:49 nl.defaults.trainer]: Epoch 4 done. Train accuracy: 99.96400, Validation accuracy: 90.21000
[10/06 13:41:49 nl.defaults.trainer]: Training finished
Train accuracies: [99.116, 99.5520000024414, 90.96799999023438, 99.16800000732422, 99.964]
Validation accuracies: [86.81, 84.74, 84.01, 84.36, 90.21]
[10/06 13:41:49 nl.defaults.trainer]: Start evaluation
[10/06 13:41:49 nl.defaults.trainer]: loading model from file runs/search/model_fina

90.21

In [21]:
def update_config(config, optimizer_type, search_space_type, dataset, seed):
    # Dataset being used
    config.dataset = dataset
    
    # Directory to which the results/logs will be saved
    config.save = f"runs/{optimizer_type.__name__}/{search_space_type.__name__}/{dataset}/{seed}"
    
    # Seed used during search phase of the optimizer
    config.search.seed = seed
    
def run_optimizer(optimizer_type, search_space_type, dataset, dataset_api, config, seed):
    # Update the config
    update_config(config, optimizer_type, search_space_type, dataset, seed)

    # Make the results directories
    os.makedirs(config.save + '/search', exist_ok=True)
    os.makedirs(config.save + '/eval', exist_ok=True)

    # Set up the loggers
    logger = setup_logger()
    logger.setLevel(logging.INFO)

     # See the config
    logger.info(f'Configuration is \n{config}')

    # Set up the seed
    utils.set_seed(seed)

    # Instantiate the search space
    n_classes = {
        'cifar10': 10,
        'cifar100': 100,
        'ImageNet16-120': 120
    }
    search_space = search_space_type(n_classes=n_classes[dataset])

    # Get the benchmark API
    logger.info('Loading Benchmark API')
    dataset_api = get_dataset_api(search_space.get_type(), dataset)
    
    # Instantiate the optimizer and adapat the search space to the optimizer
    optimizer = optimizer_type(config)
    optimizer.adapt_search_space(search_space, dataset_api=dataset_api)

    # Create a Trainer
    trainer = Trainer(optimizer, config)

    # Perform the search
    trainer.search(report_incumbent=False)
    # Get the results of the search
    search_trajectory = trainer.search_trajectory
    print('Train accuracies:', search_trajectory.train_acc)
    print('Validation accuracies:', search_trajectory.valid_acc)

    # Get the validation performance of the best model found in the search phase
    best_model_val_acc = trainer.evaluate(dataset_api=dataset_api, metric=Metric.VAL_ACCURACY)
    best_model_val_acc

    best_model = optimizer.get_final_architecture()

    return search_trajectory, best_model, best_model_val_acc

In [23]:
# Set the optimizer and search space types
# They will be instantiated inside run_optimizer
optimizer_type = RegularizedEvolution # {RegularizedEvolution, RandomSearch}
search_space_type = NasBench201SearchSpace # {NasBench101SearchSpace, NasBench201SearchSpace, NasBench301SearchSpace}

# Set the dataset
dataset = 'cifar10' # cifar10 for NB101 and NB301, {cifar100, ImageNet16-120} for NB201

# The configuration used by the Trainer and Optimizer
# The missing information will be populated inside run_optimizer
config = {
    'search': {
        # Required by Trainer
        'epochs': 100,
        'checkpoint_freq': 100,
        
        # Required by Random Search optimizer
        'fidelity': -1,
        
        # Required by RegularizedEvolution
        'sample_size': 10,
        'population_size': 30,
    }
}
config = CfgNode.load_cfg(json.dumps(config))

search_trajectory, best_model, best_model_val_acc = run_optimizer(
                                                        optimizer_type,
                                                        search_space_type,
                                                        dataset,
                                                        dataset_api,
                                                        config,
                                                        9001
                                                    )

[10/06 13:43:15 naslib]: Configuration is 
dataset: cifar10
save: runs/RegularizedEvolution/NasBench201SearchSpace/cifar10/9001
search:
  checkpoint_freq: 100
  epochs: 100
  fidelity: -1
  population_size: 30
  sample_size: 10
  seed: 9001
[10/06 13:43:15 naslib]: Loading Benchmark API
[10/06 13:43:19 nl.defaults.trainer]: param size = 0.000000MB
[10/06 13:43:19 nl.defaults.trainer]: Start training
[10/06 13:43:19 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[10/06 13:43:19 nl.optimizers.discrete.re.optimizer]: Population size 1
[10/06 13:43:19 nl.defaults.trainer]: Epoch 0 done. Train accuracy: 79.06800, Validation accuracy: 74.56000
[10/06 13:43:19 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the population
[10/06 13:43:19 nl.defaults.trainer]: Epoch 1 done. Train accuracy: 97.60000, Validation accuracy: 85.57000
[10/06 13:43:19 nl.optimizers.discrete.re.optimizer]: Start sampling architectures to fill the po